# 데이터 병합작업

데이터 전처리가 완료된 파일들을 하나의 데이터 프레임으로 만들기 위한 작업.
1. 각 `csv 파일`마다 최종적으로 컬럼을 선별하고 정제
2. `구분` 컬럼을 추가하여 인덱스로 잡고 모든 파일을 병합

## 라이브러리 및 경로 세팅

In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 100, "display.max_columns", 100)

import warnings
warnings.filterwarnings('ignore')

In [2]:
# PATH = 'D:/project/데이터 분석 프로젝트/이노포스트/data/'
PATH = 'G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data/'


In [3]:
# 구글 드라이브 마운트
# from google.colab import drive
# drive.mount('/content/drive')

## 데이터 로드

In [4]:
fine = pd.read_csv(PATH + '02_과태료/과태료_장병용_전처리_행정동 추출(2805).csv', encoding='cp949')
nszone = pd.read_csv(PATH + '03_금연구역/금연구역_장병용_전처리_행정동추출(44475).csv', encoding='cp949')
hospital = pd.read_csv(PATH + '05_병의원/병의원_박민서_행정동추출_1202.csv', encoding='cp949')
parking = pd.read_csv(PATH + '06_주차장/주차장_장병용_전처리_행정동 추출(850).csv', encoding='cp949')
comm = pd.read_csv(PATH + '07_상권정보/상권정보_장병용_전처리_1202.csv', encoding='cp949')
survey = pd.read_csv(PATH + '08_지역사회건강조사/2021 지역사회건강조사_흡연과련문항_전처리.csv', encoding='cp949')
population = pd.read_csv(PATH + '09_주민등록인구/주민등록인구_장병용_10단위_1205.csv', encoding='cp949')

print('done')

done


In [5]:
# 데이터 별 shape 확인
fine.shape, nszone.shape, hospital.shape, parking.shape, comm.shape, population.shape

((2805, 10), (44475, 11), (2300, 10), (850, 11), (75145, 13), (164, 13))

## 컬럼 정제 작업

### 과태료

In [6]:
fine.head(2)

부과번호          부과대상        부과일자      최종납기일자   최초본세    최종본세  \
0  2014-000001-00   서구 둔산로 223   2014-08-12  2014-08-27  80000  80,000   
1  2014-000002-00  서구 둔산북로 160   2014-08-12  2014-08-27  80000  80,000   

          위도         경도   구   행정동  
0  36.351933  127.39813  서구  둔산3동  
1  36.355610  127.39083  서구  둔산1동

In [7]:
# 구분 인덱스 추가
fine.insert(0, '구분', '과태료')

In [8]:
fine = fine[['구분',
             '구',
             '행정동',
             '경도',
             '위도',
             '부과일자',
             '최초본세']]

#fine.rename(columns = {})

fine.head(2)

구분   구   행정동         경도         위도        부과일자   최초본세
0  과태료  서구  둔산3동  127.39813  36.351933  2014-08-12  80000
1  과태료  서구  둔산1동  127.39083  36.355610  2014-08-12  80000

In [9]:
# csv 저장
fine.to_csv(PATH + '전처리 완료 데이터/과태료_전처리(완)_1205.csv', index=False, encoding='cp949')

### 금연구역

In [10]:
nszone.head(2)

구        구분    시설명         주소                         번지주소  세부주소  \
0  중구  학교교과교습학원  중앙독서실   목중로26번길          대전광역시 중구 목중로26번길 83    83    
1  중구    게임제공업소  지존PC방  목중로54번길 2  대전광역시 중구 목중로54번길 2 2층 (중촌동)   2층    

             kakao 주소    동         위도          경도   행정동  
0    중구 목중로26번길 중앙독서실  중촌동  36.338707  127.414436   중촌동  
1  중구 목중로54번길 2 지존PC방  판암동  36.324284  127.458253  판암2동

In [11]:
# 컬럼명 변경
nszone.rename(columns = {'구분':'분류', '시설명':'이름'}, inplace=True)

In [12]:
# 구분 인덱스 추가
nszone.insert(0, '구분', '금연구역')

In [13]:
nszone = nszone[['구분',
                 '구',
                 '행정동',
                 '경도',
                 '위도',
                 '분류',
                 '이름']]

nszone.head(2)

구분   구   행정동          경도         위도        분류     이름
0  금연구역  중구   중촌동  127.414436  36.338707  학교교과교습학원  중앙독서실
1  금연구역  중구  판암2동  127.458253  36.324284    게임제공업소  지존PC방

In [14]:
# csv 파일 저장
nszone.to_csv(PATH + '전처리 완료 데이터/금연구역_전처리(완)_1205.csv', index=False, encoding='cp949')

### 병의원

In [15]:
hospital.head(2)


index        요양기관명 종별코드명 시군구코드명    동                      주소   우편번호  \
0      0      충남대학교병원  상급종합   대전중구  대사동  대전광역시 중구 문화로 282 (대사동)  35015   
1      1  근로복지공단 대전병원  종합병원  대전대덕구   법동  대전광역시 대덕구 계족로 637 (법동)  34384   

          x좌표        y좌표  행정동  
0  127.415363  36.316908  대사동  
1  127.428549  36.368398  법1동

In [16]:
# 구분 인덱스 추가
hospital.insert(0, '구분', '병의원')

In [17]:
# 대전 데이터만 추출
hospital['시군구코드명'] = hospital['시군구코드명'].str.replace(pat='대전', repl='')

In [18]:
# 컬럼명 변경
hospital.rename(columns = {'시군구코드명':'구', 'x좌표':'경도', 'y좌표':'위도',  '종별코드명':'병원유형', '요양기관명':'이름'}, inplace=True)

In [19]:
# 필요한 컬럼만 지정
hospital = hospital[['구분',
                     '구',
                     '행정동',
                     '경도',                    
                     '위도',
                     '병원유형',
                     '이름']]

hospital.head(2)

구분    구  행정동          경도         위도  병원유형           이름
0  병의원   중구  대사동  127.415363  36.316908  상급종합      충남대학교병원
1  병의원  대덕구  법1동  127.428549  36.368398  종합병원  근로복지공단 대전병원

In [20]:
# csv 저장
hospital.to_csv(PATH + '전처리 완료 데이터/병의원_전처리(완)_1205.csv', index=False, encoding='cp949')

### 주차장

In [21]:
parking.head(2)

주차장명 주차장유형         소재지도로명주소                 소재지지번주소         위도  \
0  한약·인쇄거리 노외공영주차장    노외  대전광역시 동구 태전로 27  대전광역시 동구 중동 9-5, 10-16  36.332905   
1   선화교 하상주차장(삼성동)    노외   대전광역시 동구 대전천동로            대전광역시 동구 대전천  36.314490   

           경도  제공기관명      시   구 법정동  행정동  
0  127.429434  대전광역시  대전광역시  동구  중동  중앙동  
1  127.439904  대전광역시  대전광역시  동구  효동   효동

In [22]:
parking.insert(0, '구분', '주차장')

In [23]:
parking =parking[['구분',
                  '구',
                  '행정동',
                  '경도',
                  '위도',
                  '주차장유형'
                  ]]
parking.head(2)

구분   구  행정동          경도         위도 주차장유형
0  주차장  동구  중앙동  127.429434  36.332905    노외
1  주차장  동구   효동  127.439904  36.314490    노외

In [24]:
parking.to_csv(PATH + '전처리 완료 데이터/주차장_전처리(완)_1205.csv', index=False, encoding='cp949')

### 상권정보

In [25]:
comm.head(2)

Unnamed: 0      상호-지점명 상권업종대분류명 상권업종중분류명   상권업종소분류명    시도명 시군구명  행정동명 법정동명  \
0           0      스피드메이트    생활서비스  자동차/이륜차  자동차정비/카센타  대전광역시  유성구   관평동  관평동   
1           1  국제종합기계부품총판       소매    기타판매업      농기구판매  대전광역시   서구  도마2동  도마동   

                 지번주소                           도로명주소          경도         위도  
0  대전광역시 유성구 관평동 1122        대전광역시 유성구 관들4길 25, (관평동)  127.393241  36.424448  
1  대전광역시 서구 도마동 173-1  대전광역시 서구 계백로 1426, (도마동, 써머스빌)  127.379765  36.312625

In [26]:
comm.insert(0, '구분', '상권정보')

In [27]:
comm.rename(columns = {'상호-지점명':'이름',
                       '상권업종대분류명':'대분류',
                       '상권업종중분류명':'중분류',
                       '상권업종소분류명':'소분류',
                       '시군구명':'구',
                       '행정동명':'행정동'
                       }, inplace=True)

In [28]:
comm = comm[['구분', 
             '구', 
             '행정동', 
             '경도', 
             '위도', 
             '이름', 
             '대분류', 
             '중분류', 
             '소분류'
             ]]
comm.head(2)

구분    구   행정동          경도         위도          이름    대분류      중분류  \
0  상권정보  유성구   관평동  127.393241  36.424448      스피드메이트  생활서비스  자동차/이륜차   
1  상권정보   서구  도마2동  127.379765  36.312625  국제종합기계부품총판     소매    기타판매업   

         소분류  
0  자동차정비/카센타  
1      농기구판매

In [29]:
comm.to_csv(PATH + '전처리 완료 데이터/상권정보_전처리(완)_1205.csv', index=False, encoding='cp949')

### 주민등록인구

In [30]:
population.head(2)

Unnamed: 0                     index   구  행정동  성별(0남자1여자)  total  0~9세  \
0           1  대전광역시 동구 중앙동(3011051500)  동구  중앙동           0   2370    45   
1           2   대전광역시 동구 효동(3011053000)  동구   효동           0  11799  1005   

   10~19세  20~29세  30~39세  40~49세  50~59세  60 over  
0      50     170     197     282     531     1095  
1    1719    1320    1191    2277    1833     2454

In [31]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  164 non-null    int64 
 1   index       164 non-null    object
 2   구           164 non-null    object
 3   행정동         164 non-null    object
 4   성별(0남자1여자)  164 non-null    int64 
 5   total       164 non-null    int64 
 6   0~9세        164 non-null    int64 
 7   10~19세      164 non-null    int64 
 8   20~29세      164 non-null    int64 
 9   30~39세      164 non-null    int64 
 10  40~49세      164 non-null    int64 
 11  50~59세      164 non-null    int64 
 12  60 over     164 non-null    int64 
dtypes: int64(10), object(3)
memory usage: 16.8+ KB


In [32]:
population.insert(0, '구분', '인구')

In [33]:
# 성별 코드를 남자 / 여자로 변경
population['성별(0남자1여자)'] = population['성별(0남자1여자)'].replace(0, '남자')
population['성별(0남자1여자)'] = population['성별(0남자1여자)'].replace(1, '여자')
population.head(2)

구분  Unnamed: 0                     index   구  행정동 성별(0남자1여자)  total  0~9세  \
0  인구           1  대전광역시 동구 중앙동(3011051500)  동구  중앙동         남자   2370    45   
1  인구           2   대전광역시 동구 효동(3011053000)  동구   효동         남자  11799  1005   

   10~19세  20~29세  30~39세  40~49세  50~59세  60 over  
0      50     170     197     282     531     1095  
1    1719    1320    1191    2277    1833     2454

In [34]:
# 값이 '남자'와 '여자'만 남았는지 확인
population['성별(0남자1여자)'].unique()

array(['남자', '여자'], dtype=object)

In [35]:
population.rename(columns = {'성별(0남자1여자)':'성별'}, inplace=True)

In [36]:
# 인구수를 20단위로 통합
a = population.copy()
a['0~19세'] = a['0~9세']+a['10~19세']
a['20~39세'] = a['20~29세']+a['30~39세']
a['40~59세'] = a['40~49세']+a['50~59세']
a.head(2)

구분  Unnamed: 0                     index   구  행정동  성별  total  0~9세  10~19세  \
0  인구           1  대전광역시 동구 중앙동(3011051500)  동구  중앙동  남자   2370    45      50   
1  인구           2   대전광역시 동구 효동(3011053000)  동구   효동  남자  11799  1005    1719   

   20~29세  30~39세  40~49세  50~59세  60 over  0~19세  20~39세  40~59세  
0     170     197     282     531     1095     95     367     813  
1    1320    1191    2277    1833     2454   2724    2511    4110

In [37]:
population = a[['구분',
                '구', 
                '행정동', 
                '성별', 
                'total', 
                '0~19세', 
                '20~39세', 
                '40~59세',
                '60 over']]
population.head(2)

구분   구  행정동  성별  total  0~19세  20~39세  40~59세  60 over
0  인구  동구  중앙동  남자   2370     95     367     813     1095
1  인구  동구   효동  남자  11799   2724    2511    4110     2454

In [38]:
population.to_csv(PATH + '전처리 완료 데이터/주민등록인구_전처리(완)_1205.csv', index=False, encoding='cp949')

### 지역사회건강조사

In [39]:
survey.head(3)

시도번호  나이  성별  동/읍면 구분 행정기관  일반담배 평생 흡연 여부  일반담배 현재 흡연 여부  \
0    30  70   1        1  대덕구              2              1   
1    30  66   2        1  대덕구              3              8   
2    30  49   2        1  대덕구              3              8   

   일반담배 매일 흡연자의 하루평균 흡연량  평생 니코틴이 포함된 액상전자담배 사용 여부  현재 니코틴이 포함된 액상전자담배 사용 여부  \
0                     10                         2                         8   
1                    888                         2                         8   
2                    888                         2                         8   

   현재 흡연자의 금연 시도  현재 흡연자의 금연 계획  가정실내 간접흡연 여부  가정실내 간접흡연 경험 여부  \
0              2              4             2                8   
1              8              8             2                8   
2              8              8             2                8   

   직장실내 간접흡연 경험 여부  공공장소 간접흡연 경험 여부              설명  
0                3                2      * 7 = 응답거부  
1                2                2  * 8, 888 = 비해당  
2                3                2        * 9 = 모름

In [40]:
# 성별 코드를 남자 / 여자로 변경
survey['성별'] = survey['성별'].replace(2, '여자')
survey['성별'] = survey['성별'].replace(1, '남자')
survey.head(2)

시도번호  나이  성별  동/읍면 구분 행정기관  일반담배 평생 흡연 여부  일반담배 현재 흡연 여부  \
0    30  70  남자        1  대덕구              2              1   
1    30  66  여자        1  대덕구              3              8   

   일반담배 매일 흡연자의 하루평균 흡연량  평생 니코틴이 포함된 액상전자담배 사용 여부  현재 니코틴이 포함된 액상전자담배 사용 여부  \
0                     10                         2                         8   
1                    888                         2                         8   

   현재 흡연자의 금연 시도  현재 흡연자의 금연 계획  가정실내 간접흡연 여부  가정실내 간접흡연 경험 여부  \
0              2              4             2                8   
1              8              8             2                8   

   직장실내 간접흡연 경험 여부  공공장소 간접흡연 경험 여부              설명  
0                3                2      * 7 = 응답거부  
1                2                2  * 8, 888 = 비해당

In [41]:
survey['성별'].unique()

array(['남자', '여자'], dtype=object)

In [42]:
survey.insert(0, '구분', '설문')

In [43]:
survey.rename(columns = {'일반담배 현재 흡연 여부':'담배흡연', 
                                   '현재 니코틴이 포함된 액상전자담배 사용 여부':'액상흡연',
                                   '행정기관':'구',
                                   '동/읍면 구분':'행정동'
                                   }, inplace=True)

In [44]:
survey.head(2)

구분  시도번호  나이  성별  행정동    구  일반담배 평생 흡연 여부  담배흡연  일반담배 매일 흡연자의 하루평균 흡연량  \
0  설문    30  70  남자    1  대덕구              2     1                     10   
1  설문    30  66  여자    1  대덕구              3     8                    888   

   평생 니코틴이 포함된 액상전자담배 사용 여부  액상흡연  현재 흡연자의 금연 시도  현재 흡연자의 금연 계획  가정실내 간접흡연 여부  \
0                         2     8              2              4             2   
1                         2     8              8              8             2   

   가정실내 간접흡연 경험 여부  직장실내 간접흡연 경험 여부  공공장소 간접흡연 경험 여부              설명  
0                8                3                2      * 7 = 응답거부  
1                8                2                2  * 8, 888 = 비해당

In [45]:
survey['담배흡연'].unique()

array([1, 8, 3, 2], dtype=int64)

In [46]:
survey = survey[['구분',
                 '구',
                 '행정동',
                 '성별',
                 '담배흡연',
                 '액상흡연']]
survey.head(2)

구분    구  행정동  성별  담배흡연  액상흡연
0  설문  대덕구    1  남자     1     8
1  설문  대덕구    1  여자     8     8

In [47]:
survey.to_csv(PATH + '전처리 완료 데이터/지역사회건강조사_전처리(완)_1205.csv', index=False, encoding='cp949')

## EDA를 위한 데이터 프레임 병합


In [49]:
# PATH2 = 'D:/project/데이터 분석 프로젝트/이노포스트/data/전처리 완료 데이터/'
PATH2 = 'G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data/전처리 완료 데이터/'

fine = pd.read_csv(PATH2 + '과태료_전처리(완)_1205.csv', encoding='cp949')
nszone = pd.read_csv(PATH2 + '금연구역_전처리(완)_1205.csv', encoding='cp949')
hospital = pd.read_csv(PATH2 + '병의원_전처리(완)_1205.csv', encoding='cp949')
parking = pd.read_csv(PATH2 + '주차장_전처리(완)_1205.csv', encoding='cp949')
comm = pd.read_csv(PATH2 + '상권정보_전처리(완)_1205.csv', encoding='cp949')
survey = pd.read_csv(PATH2 + '지역사회건강조사_전처리(완)_1205.csv', encoding='cp949')
population = pd.read_csv(PATH2 + '주민등록인구_전처리(완)_1205.csv', encoding='cp949')

print('done')

done


In [50]:
fine.shape, nszone.shape, hospital.shape, parking.shape, comm.shape, survey.shape, population.shape

((2805, 7), (44475, 7), (2300, 7), (850, 6), (75145, 9), (4610, 6), (164, 9))

In [51]:
# df에 모든 컬럼을 추가함
df = pd.concat([fine, nszone], ignore_index=True)
df = pd.concat([df, hospital], ignore_index=True)
df = pd.concat([df, parking], ignore_index=True)
df = pd.concat([df, comm], ignore_index=True)
df = pd.concat([df, survey], ignore_index=True)
df = pd.concat([df, population], ignore_index=True)

#구, 행정동 기준으로 정렬
df.sort_values(by=['구', '행정동'], inplace=True, ignore_index=True)

df.shape, df.head(2)

((130349, 22),
    구분    구 행정동  경도  위도 부과일자  최초본세   분류   이름 병원유형 주차장유형  대분류  중분류  소분류  성별  \
 0  설문  대덕구   1 NaN NaN  NaN   NaN  NaN  NaN  NaN   NaN  NaN  NaN  NaN  남자   
 1  설문  대덕구   1 NaN NaN  NaN   NaN  NaN  NaN  NaN   NaN  NaN  NaN  NaN  여자   
 
    담배흡연  액상흡연  total  0~19세  20~39세  40~59세  60 over  
 0   1.0   8.0    NaN    NaN     NaN     NaN      NaN  
 1   8.0   8.0    NaN    NaN     NaN     NaN      NaN  )

### 행정동 체크

In [52]:
df['행정동'].unique()

array([1, '구즉동', '대화동', '덕암동', '목상동', '법1동', '법2동', '비래동', '석봉동', '송촌동',
       '신탄진동', '오정동', '은행선화동', '중리동', '회덕동', '가수원동', '가양1동', '가양2동',
       '관저2동', '대동', '대청동', '도안동', '둔산2동', '산내동', '삼성동', '성남동', '신인동',
       '용문동', '용운동', '용전동', '자양동', '정림동', '중앙동', '탄방동', '판암1동', '판암2동',
       '홍도동', '효동', '가장동', '갈마1동', '갈마2동', '관저1동', '관평동', '괴정동', '기성동',
       '내동', '노은3동', '대사동', '도마1동', '도마2동', '둔산1동', '둔산3동', '만년동', '문화1동',
       '변동', '복수동', '산성동', '온천1동', '온천2동', '용두동', '원신흥동', '월평1동', '월평2동',
       '월평3동', '전민동', '진잠동', '태평1동', '학하동', '노은1동', '노은2동', '상대동', '신성동',
       '광희동', '대흥동', '명동', '목동', '문창동', '문화2동', '부사동', '석교동', '성내1동',
       '소공동', '신흥동', '오류동', '유천1동', '유천2동', '을지로동', '중구동', '중촌동', '태평2동',
       '필동', nan], dtype=object)

In [53]:
# 행정동 컬럼 값 중 대전이 아닌 지역의 행정동이 포함된 것들
list = ['광희동', '명동', '성내1동', '소공동', '신흥동', '을지로동','중구동','필동']

for i in list:
    mask = df['행정동'].isin(list)
    df_null = df[~mask]
    
df.shape, df_null.shape

((130349, 22), (130277, 22))

In [54]:
df_null['행정동'].unique()

array([1, '구즉동', '대화동', '덕암동', '목상동', '법1동', '법2동', '비래동', '석봉동', '송촌동',
       '신탄진동', '오정동', '은행선화동', '중리동', '회덕동', '가수원동', '가양1동', '가양2동',
       '관저2동', '대동', '대청동', '도안동', '둔산2동', '산내동', '삼성동', '성남동', '신인동',
       '용문동', '용운동', '용전동', '자양동', '정림동', '중앙동', '탄방동', '판암1동', '판암2동',
       '홍도동', '효동', '가장동', '갈마1동', '갈마2동', '관저1동', '관평동', '괴정동', '기성동',
       '내동', '노은3동', '대사동', '도마1동', '도마2동', '둔산1동', '둔산3동', '만년동', '문화1동',
       '변동', '복수동', '산성동', '온천1동', '온천2동', '용두동', '원신흥동', '월평1동', '월평2동',
       '월평3동', '전민동', '진잠동', '태평1동', '학하동', '노은1동', '노은2동', '상대동', '신성동',
       '대흥동', '목동', '문창동', '문화2동', '부사동', '석교동', '오류동', '유천1동', '유천2동',
       '중촌동', '태평2동', nan], dtype=object)

In [55]:
# 대전 행정동이 아닌 값이 있는지 체크
df_null2 = df_null[(df_null['구분']=='금연구역')|(df_null['구분']=='인구')].reset_index()
df_null2 = df_null.groupby('행정동').sum() # 행정동으로 그룹바이

df_null2[df_null2['total'] < 1]

경도   위도  최초본세     담배흡연     액상흡연  total  0~19세  20~39세  40~59세  60 over
행정동                                                                         
1    0.0  0.0   0.0  28311.0  36083.0    0.0    0.0     0.0     0.0      0.0

데이터프레임 조작을 위해 결측치를 전부 0으로 변경

In [56]:
df_null.fillna(0, inplace=True)

In [57]:
df_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130277 entries, 0 to 130348
Data columns (total 22 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   구분       130277 non-null  object 
 1   구        130277 non-null  object 
 2   행정동      130277 non-null  object 
 3   경도       130277 non-null  float64
 4   위도       130277 non-null  float64
 5   부과일자     130277 non-null  object 
 6   최초본세     130277 non-null  float64
 7   분류       130277 non-null  object 
 8   이름       130277 non-null  object 
 9   병원유형     130277 non-null  object 
 10  주차장유형    130277 non-null  object 
 11  대분류      130277 non-null  object 
 12  중분류      130277 non-null  object 
 13  소분류      130277 non-null  object 
 14  성별       130277 non-null  object 
 15  담배흡연     130277 non-null  float64
 16  액상흡연     130277 non-null  float64
 17  total    130277 non-null  float64
 18  0~19세    130277 non-null  float64
 19  20~39세   130277 non-null  float64
 20  40~59세   130277 non-null  

In [58]:
# 메모리 관리를 위한 데이터타입 지정
df_null = df_null.astype({'경도' : np.float32,
                          '위도' : np.float32,
                          '최초본세' : np.uint32,
                          '성별' : 'category',
                          '담배흡연' : np.uint8, # 소수점 제거 후 카테고리로 변경
                          '액상흡연' : np.uint8, # 소수점 제거 후 카테고리로 변경
                          'total' : np.uint32,
                          '0~19세' : np.uint32,
                          '20~39세' : np.uint32,
                          '40~59세' : np.uint32,
                          '60 over' : np.uint16,
                          })

df_null = df_null.astype({'담배흡연':'category', '액상흡연':'category'})

df_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130277 entries, 0 to 130348
Data columns (total 22 columns):
 #   Column   Non-Null Count   Dtype   
---  ------   --------------   -----   
 0   구분       130277 non-null  object  
 1   구        130277 non-null  object  
 2   행정동      130277 non-null  object  
 3   경도       130277 non-null  float32 
 4   위도       130277 non-null  float32 
 5   부과일자     130277 non-null  object  
 6   최초본세     130277 non-null  uint32  
 7   분류       130277 non-null  object  
 8   이름       130277 non-null  object  
 9   병원유형     130277 non-null  object  
 10  주차장유형    130277 non-null  object  
 11  대분류      130277 non-null  object  
 12  중분류      130277 non-null  object  
 13  소분류      130277 non-null  object  
 14  성별       130277 non-null  category
 15  담배흡연     130277 non-null  category
 16  액상흡연     130277 non-null  category
 17  total    130277 non-null  uint32  
 18  0~19세    130277 non-null  uint32  
 19  20~39세   130277 non-null  uint32  
 20  40~5

In [59]:
df_null.head(2)

구분    구 행정동   경도   위도 부과일자  최초본세 분류 이름 병원유형 주차장유형 대분류 중분류 소분류  성별 담배흡연  \
0  설문  대덕구   1  0.0  0.0    0     0  0  0    0     0   0   0   0  남자    1   
1  설문  대덕구   1  0.0  0.0    0     0  0  0    0     0   0   0   0  여자    8   

  액상흡연  total  0~19세  20~39세  40~59세  60 over  
0    8      0      0       0       0        0  
1    8      0      0       0       0        0

In [60]:
# df 저장
df_null.to_pickle(PATH2 + 'EDA_1212.pkl')

In [62]:
# 저장이 잘 됐는지 체크
df_null = pd.read_pickle(PATH2 + 'EDA_1212.pkl')
df_null.head(2)

구분    구 행정동   경도   위도 부과일자  최초본세 분류 이름 병원유형 주차장유형 대분류 중분류 소분류  성별 담배흡연  \
0  설문  대덕구   1  0.0  0.0    0     0  0  0    0     0   0   0   0  남자    1   
1  설문  대덕구   1  0.0  0.0    0     0  0  0    0     0   0   0   0  여자    8   

  액상흡연  total  0~19세  20~39세  40~59세  60 over  
0    8      0      0       0       0        0  
1    8      0      0       0       0        0